In [2]:
#Testing the CTRP-L1000-3h data by using the Achilles-L1000-144h model
import pandas as pd
import numpy as np
import warnings
import xgboost as xgb
import pickle
import scipy as sc
from sklearn.metrics import mean_squared_error
pd.set_option('display.max_columns',15)
warnings.filterwarnings("ignore")

#Retrieving the data
def getData():
    #1.Reading the data
    sig_viability_info = pd.read_table(r"individual_validation\LINCS_CTRP_withoutdose_3h.txt",sep="	",dtype=str)
    sig_viability_info_filter = sig_viability_info[['signature','cell_line','drug','dose','time','cell_viability']]
    sig_GE_info = pd.read_csv(r"individual_validation\LINCS_gene_expression_3h.csv",sep=",",dtype=str)
    sig_GE_info_filter = sig_GE_info[:]

    #2.Combining the two datasets
    sig_viability = pd.merge(sig_viability_info_filter,sig_GE_info_filter,left_on='signature',right_on='cid')
    select_sig_viavility2 = pd.DataFrame(sig_viability).loc[:,'780':]
    select_sig_viavility_arr = select_sig_viavility2.copy()
    
    #3.Getting the cell viability value
    viability_value = pd.DataFrame(sig_viability).loc[:,'cell_viability']
    
    #4.Return the gene expression matrix and cell viability value
    return select_sig_viavility_arr,viability_value
    
#Retrieving the data
gene_expression,cell_viability = getData()

#Selecting the genes
selected_gene = {'26136': 1.4156162167257094, '8870': 6.753151349822873, '1514': 7.943744144483807, '2778': 10.606209011386643, '8624': 15.078081083628545, '55324': 16.865663060855262, '6676': 20.596989182565782, '11041': 28.606209011386646, '2745': 28.990780171179136, '29928': 30.29388467687246, '1062': 41.043650456224704, '10057': 42.212418022773285, '23210': 42.58438378327429, '3206': 43.53734775657896, '4690': 51.090686482920034, '58497': 53.419001787196336, '4216': 54.043650456224704, '51070': 61.68767566548581, '9448': 62.13095136867411, '1509': 62.37780001885124, '51005': 63.081466654099174, '1052': 64.55917298469132, '9928': 66.2284089925354, '5829': 66.98156034235828, '2767': 67.1435567679656, '22883': 69.25023422064776, '5927': 74.32154416333505, '79947': 74.75315134982289, '11344': 74.80940720533907, '664': 76.35352610285928, '80212': 76.49953155870449, '9903': 77.69689549430669, '9817': 78.41561621672571, '3930': 82.03104505693322, '4616': 82.94712971495443, '8520': 84.97817477188765, '6919': 85.67845583666495, '5747': 86.63386849784924, '10813': 92.58776935374492, '9217': 96.78081083628545, '7158': 101.51213695799598, '22823': 103.5431820149292, '29911': 104.79003066510631, '5720': 106.35258922026827, '11004': 107.44666127365892, '30': 110.88165403061737, '501': 111.35936036120953, '1983': 114.06209011386643, '4850': 116.85982880250502, '8996': 119.27450813663971, '23139': 119.57177838398279, '9183': 120.91853334590081, '4793': 121.44327570318829, '39': 124.67845583666495, '9797': 126.33753513309716, '11168': 126.39040541814273, '2274': 127.43160718648784, '10099': 133.56839281351216, '5997': 133.81617834628034, '9133': 134.52135678681685, '3775': 134.60037475303642, '7077': 134.81185589321868, '5993': 135.4442125857793, '9375': 136.06209011386642, '8884': 136.12173153985327, '25825': 138.10574057009111, '211': 138.32248104592608, '23223': 140.13188825126514, '6119': 142.44666127365892, '23131': 146.22502342206477, '3988': 147.04703602669534, '55127': 147.31232433451416, '55556': 149.8429009501519, '51203': 151.03198193952423, '3725': 151.0713099426873, '466': 152.43744144483807, '56654': 154.5407333270496, '10682': 160.85982880250504, '6117': 169.14939102631584, '230': 169.24778553276815, '5836': 172.78758197722672, '3028': 173.8176901515689, '6832': 174.55672429681172, '7153': 176.79680180604757, '891': 177.1285026807945, '332': 178.52567923987849, '10559': 180.9446810270748, '9924': 181.67262157831473, '1026': 183.79435311816798, '1738': 185.21580359324392, '10489': 185.21918916371453, '5290': 187.12173153985327, '148022': 188.35842347861853, '51097': 191.28372796546057, '7264': 193.76330806123477, '91949': 194.54750446799085, '4927': 195.2250234220648, '25793': 196.79003066510631, '2146': 198.23762882135628, '7866': 198.60376032350706, '375346': 199.64985946761135, '27346': 200.0208883455213, '26227': 201.6566306085526, '10775': 202.7303892391195, '2946': 204.34581807932702, '4891': 205.06547568433706, '7088': 205.20319819395243, '1021': 205.27544501923074, '5601': 205.31477302239378, '1759': 205.5416702096406, '51026': 205.69106123595643, '1027': 208.87920534273778, '81533': 211.166318878669, '392': 212.08391534197878, '3329': 212.43405587436746, '147179': 212.4457243910679, '10494': 213.77403969534421, '6182': 217.03198193952423, '8550': 220.69444680642707, '7048': 221.24439996229754, '3895': 221.53396218610834, '701': 223.15277659678648, '965': 223.47525764271253, '11200': 226.29482155946351, '9653': 227.37290264309203, '3385': 229.35014053238865, '9833': 231.92285579896247, '5480': 232.61636572279852, '2037': 232.77591346052625, '84722': 233.64985946761138, '1399': 234.114960398912, '4846': 234.52135678681682, '8878': 235.13678562702435, '84617': 235.27544501923074, '51001': 236.82294948722154, '23212': 238.11683416409403, '6616': 239.23179456300605, '58533': 240.15522528466607, '6790': 240.25174602593634, '23386': 240.6968954943067, '80347': 241.00921982882087, '30001': 241.2133549053643, '8985': 241.42483604554658, '10451': 241.70705220571855, '9868': 242.74054595053138, '7043': 248.13678562702435, '9275': 250.2745081366397, '79143': 253.16048462031873, '8440': 254.62371178643735, '993': 256.6581424138412, '8508': 257.0839153419788, '4893': 259.58193509539467, '9517': 260.07224682527834, '55148': 260.24778553276815, '4860': 260.4694233843623, '27242': 263.6658504373735, '10523': 266.231794563006, '10513': 267.46754961918026, '256364': 268.60959458185727, '6944': 269.10574057009114, '7165': 270.3476918445091, '2017': 271.6818414071356, '1050': 271.87488288967614, '226': 272.3283153042763, '2115': 272.884102718497, '64746': 272.9016054935477, '5883': 273.5247423572875, '10227': 276.74148283312235, '26036': 276.90931351708, '25839': 278.4835405889424, '10668': 282.0882377950404, '6194': 282.8152414636893, '4775': 284.03687931528344, '3682': 284.0756323957489, '2048': 286.2114811401823, '9801': 286.61053146444834, '23224': 286.97572608400804, '2264': 287.114960398912, '1846': 287.2711225661691, '8573': 287.8229494872216, '10589': 288.03781619787446, '5427': 288.41071884096647, '6696': 289.0679243722167, '4817': 289.1838216537197, '5899': 289.3249297338057, '2288': 289.40545950531384, '51160': 291.56594412563254, '3122': 292.2459117675861, '8826': 292.3123243345142, '840': 296.93697300354256, '79921': 298.4777063305921, '6908': 299.0999063117409, '5925': 300.9461928323634, '2771': 302.04458733881575, '4609': 302.8530576615638, '93487': 304.10422876480254, '80349': 304.98494591282895, '890': 305.3418575861588, '16': 305.60714589397764, '5777': 306.3399838209768, '63933': 306.90160549354766, '64781': 309.44817307894755, '1906': 309.6590792964322, '1070': 312.0193765402327, '3300': 314.5475044679908, '5300': 315.92343072166005, '128': 316.5951154173838, '2185': 317.7608593733552, '6009': 318.26867387828946, '2523': 319.47525764271256, '10972': 320.9428072618928, '994': 321.53979644445855, '899': 321.6105314644483, '23244': 322.4238991629556, '9124': 322.8923856647269, '23338': 323.344881196736, '23647': 324.16048462031875, '10681': 324.8220126046306, '9552': 328.28221616017197, '4794': 328.71382334665986, '9276': 328.9160846580212, '10285': 329.25361979111835, '10644': 329.4238991629556, '23443': 329.47338387753047, '4582': 329.84816028580457, '1831': 330.09068648292003, '11188': 330.7817477188765, '9650': 331.7288774338309, '3280': 333.3855080423835, '9943': 333.8278468629808, '10915': 334.12850268079455, '54541': 334.29727024734314, '54623': 334.46020355554145, '1891': 334.59605229997476, '3066': 335.60714589397764, '5058': 335.75898560817313, '5986': 338.71101269888675, '7015': 339.1076143352731, '9847': 339.7017928700659, '29103': 340.0141172045801, '9212': 340.1294395633855, '11065': 340.3821224719129, '28969': 342.8545694668524, '5261': 344.9151477754302, '65123': 344.99229197646775, '23321': 345.3768631362603, '55608': 346.1450685732542, '2736': 346.3904054181427, '1677': 346.48109190106277, '2852': 346.9763010067056, '6709': 347.65418192067295, '29937': 347.97572608400804, '7398': 348.08823779504047, '5710': 348.63141980996966, '8349': 349.7565369202935, '23512': 350.1905927946609, '3553': 350.3157099049848, '51021': 350.320607280744, '9533': 350.6381909509109, '5880': 350.64496209185216, '808': 351.60037475303636, '60493': 352.4447875084769, '1454': 353.9379098861336, '85377': 354.0829784593878, '4303': 354.6255855516194, '5627': 355.4786432131831, '3312': 355.8767566548582, '5373': 356.49520910564286, '1662': 357.82013883944853, '2920': 359.7608593733552, '8851': 360.72304317548065, '7485': 361.23518013347666, '57215': 362.08636402985843, '1017': 362.7866450946357, '10362': 362.9582233089573, '5357': 364.164445113487, '22926': 364.487863042004, '5583': 365.19642705301123, '5898': 365.8220126046306, '5223': 367.4558811024798, '51056': 367.6329316152582, '8607': 368.67751895407395, '80204': 369.72455498076926, '1845': 370.72981431642194, '2063': 370.8395153796813, '54512': 371.4238991629556, '23588': 371.47864321318315, '29082': 372.2895622238108, '5111': 372.75163954453427, '949': 373.11928285197365, '8061': 373.52661612246953, '23530': 374.115897281503, '5440': 375.65569372596156, '7319': 375.7405459505314, '23326': 375.8118558932187, '7168': 375.8724342017965, '23670': 375.97478920141697, '6390': 376.0882377950404, '11284': 376.42051359248495, '22889': 378.7851332893471, '79071': 381.6397027561994, '10953': 381.67262157831476, '64422': 382.6643386320849, '9531': 387.4389532501267, '57761': 387.7202325277076, '3480': 387.9093135170799, '66008': 390.3006558178137, '7994': 390.35842347861853, '6804': 390.91947022849183, '80758': 391.1048036875001, '4605': 391.19455328782914, '1870': 391.37967378403323, '2956': 391.6430883266701, '6500': 391.9539008558957, '23011': 392.57667575974205, '9712': 392.9326505504809, '10670': 393.2022613113614, '6347': 394.11834596938263, '7106': 394.4733838775305, '90861': 395.2585171668776, '3251': 395.3049782708753, '57804': 395.8801422253288, '5715': 396.5945404946862, '124583': 398.03932800316306, '3098': 400.32248104592605, '9016': 400.5238054746964, '51719': 400.6824163298332, '79850': 400.7328379269991, '154': 401.1512647914979, '9467': 401.6649135547824, '7358': 404.24836045546573, '4172': 405.2904991064018, '6772': 405.93301251037434, '11044': 406.40056212955466, '51335': 406.96463249000516, '8321': 407.41806490460533, '2770': 407.79529000075905, '8800': 407.87056043661454, '5699': 408.42332424025795, '5696': 408.6188144106781, '3693': 409.4506217668271, '51031': 409.4626522434211, '5708': 410.0135422818825, '6774': 410.257005361589, '1788': 410.6726215783147, '6777': 411.16970444913954, '2263': 412.39134230073375, '5721': 412.44759815624997, '5921': 412.5038540117661, '8446': 412.6882505881834}
gene_expression_filter = np.array(gene_expression[list(selected_gene.keys())],dtype="float32")
cell_viability = np.array(cell_viability,dtype="float32")
dtest = xgb.DMatrix(gene_expression_filter,cell_viability)

#Loading model
loaded_model = pickle.load(open("model_adjust\Achilles-L1000-144h_XGBoost_1.dat","rb"))
print("Model Loaded Successfully!")

#Predicting the cell viability
cl_prediction = loaded_model.predict(dtest)
print('Test_Best_Pearson:',sc.stats.pearsonr(cl_prediction,cell_viability))
print("MSE_Test: %.3f" % (mean_squared_error(cl_prediction,cell_viability)))

Model Loaded Successfully!
Test_Best_Pearson: (0.07258464, 0.014667512704496951)
MSE_Test: 1.917


In [3]:
#Testing the CTRP-L1000-6h data by using the Achilles-L1000-144h model
import pandas as pd
import numpy as np
import warnings
import xgboost as xgb
import pickle
import scipy as sc
from sklearn.metrics import mean_squared_error
pd.set_option('display.max_columns',15)
warnings.filterwarnings("ignore")

#Retrieving the data
def getData():
    #1.Reading the data
    sig_viability_info = pd.read_table(r"individual_validation\LINCS_CTRP_withoutdose_6h.txt",sep="	",dtype=str)
    sig_viability_info_filter = sig_viability_info[['signature','cell_line','drug','dose','time','cell_viability']]
    sig_GE_info = pd.read_csv(r"individual_validation\LINCS_gene_expression_6h.csv",sep=",",dtype=str)
    sig_GE_info_filter = sig_GE_info[:]

    #2.Combining the two datasets
    sig_viability = pd.merge(sig_viability_info_filter,sig_GE_info_filter,left_on='signature',right_on='cid')
    select_sig_viavility2 = pd.DataFrame(sig_viability).loc[:,'5720':]
    select_sig_viavility_arr = select_sig_viavility2.copy()
    
    #3.Getting the cell viability value
    viability_value = pd.DataFrame(sig_viability).loc[:,'cell_viability']
    
    #4.Return the gene expression matrix and cell viability value
    return select_sig_viavility_arr,viability_value
    
#Retrieving the data
gene_expression,cell_viability = getData()

#Selecting the genes
selected_gene = {'26136': 1.4156162167257094, '8870': 6.753151349822873, '1514': 7.943744144483807, '2778': 10.606209011386643, '8624': 15.078081083628545, '55324': 16.865663060855262, '6676': 20.596989182565782, '11041': 28.606209011386646, '2745': 28.990780171179136, '29928': 30.29388467687246, '1062': 41.043650456224704, '10057': 42.212418022773285, '23210': 42.58438378327429, '3206': 43.53734775657896, '4690': 51.090686482920034, '58497': 53.419001787196336, '4216': 54.043650456224704, '51070': 61.68767566548581, '9448': 62.13095136867411, '1509': 62.37780001885124, '51005': 63.081466654099174, '1052': 64.55917298469132, '9928': 66.2284089925354, '5829': 66.98156034235828, '2767': 67.1435567679656, '22883': 69.25023422064776, '5927': 74.32154416333505, '79947': 74.75315134982289, '11344': 74.80940720533907, '664': 76.35352610285928, '80212': 76.49953155870449, '9903': 77.69689549430669, '9817': 78.41561621672571, '3930': 82.03104505693322, '4616': 82.94712971495443, '8520': 84.97817477188765, '6919': 85.67845583666495, '5747': 86.63386849784924, '10813': 92.58776935374492, '9217': 96.78081083628545, '7158': 101.51213695799598, '22823': 103.5431820149292, '29911': 104.79003066510631, '5720': 106.35258922026827, '11004': 107.44666127365892, '30': 110.88165403061737, '501': 111.35936036120953, '1983': 114.06209011386643, '4850': 116.85982880250502, '8996': 119.27450813663971, '23139': 119.57177838398279, '9183': 120.91853334590081, '4793': 121.44327570318829, '39': 124.67845583666495, '9797': 126.33753513309716, '11168': 126.39040541814273, '2274': 127.43160718648784, '10099': 133.56839281351216, '5997': 133.81617834628034, '9133': 134.52135678681685, '3775': 134.60037475303642, '7077': 134.81185589321868, '5993': 135.4442125857793, '9375': 136.06209011386642, '8884': 136.12173153985327, '25825': 138.10574057009111, '211': 138.32248104592608, '23223': 140.13188825126514, '6119': 142.44666127365892, '23131': 146.22502342206477, '3988': 147.04703602669534, '55127': 147.31232433451416, '55556': 149.8429009501519, '51203': 151.03198193952423, '3725': 151.0713099426873, '466': 152.43744144483807, '56654': 154.5407333270496, '10682': 160.85982880250504, '6117': 169.14939102631584, '230': 169.24778553276815, '5836': 172.78758197722672, '3028': 173.8176901515689, '6832': 174.55672429681172, '7153': 176.79680180604757, '891': 177.1285026807945, '332': 178.52567923987849, '10559': 180.9446810270748, '9924': 181.67262157831473, '1026': 183.79435311816798, '1738': 185.21580359324392, '10489': 185.21918916371453, '5290': 187.12173153985327, '148022': 188.35842347861853, '51097': 191.28372796546057, '7264': 193.76330806123477, '91949': 194.54750446799085, '4927': 195.2250234220648, '25793': 196.79003066510631, '2146': 198.23762882135628, '7866': 198.60376032350706, '375346': 199.64985946761135, '27346': 200.0208883455213, '26227': 201.6566306085526, '10775': 202.7303892391195, '2946': 204.34581807932702, '4891': 205.06547568433706, '7088': 205.20319819395243, '1021': 205.27544501923074, '5601': 205.31477302239378, '1759': 205.5416702096406, '51026': 205.69106123595643, '1027': 208.87920534273778, '81533': 211.166318878669, '392': 212.08391534197878, '3329': 212.43405587436746, '147179': 212.4457243910679, '10494': 213.77403969534421, '6182': 217.03198193952423, '8550': 220.69444680642707, '7048': 221.24439996229754, '3895': 221.53396218610834, '701': 223.15277659678648, '965': 223.47525764271253, '11200': 226.29482155946351, '9653': 227.37290264309203, '3385': 229.35014053238865, '9833': 231.92285579896247, '5480': 232.61636572279852, '2037': 232.77591346052625, '84722': 233.64985946761138, '1399': 234.114960398912, '4846': 234.52135678681682, '8878': 235.13678562702435, '84617': 235.27544501923074, '51001': 236.82294948722154, '23212': 238.11683416409403, '6616': 239.23179456300605, '58533': 240.15522528466607, '6790': 240.25174602593634, '23386': 240.6968954943067, '80347': 241.00921982882087, '30001': 241.2133549053643, '8985': 241.42483604554658, '10451': 241.70705220571855, '9868': 242.74054595053138, '7043': 248.13678562702435, '9275': 250.2745081366397, '79143': 253.16048462031873, '8440': 254.62371178643735, '993': 256.6581424138412, '8508': 257.0839153419788, '4893': 259.58193509539467, '9517': 260.07224682527834, '55148': 260.24778553276815, '4860': 260.4694233843623, '27242': 263.6658504373735, '10523': 266.231794563006, '10513': 267.46754961918026, '256364': 268.60959458185727, '6944': 269.10574057009114, '7165': 270.3476918445091, '2017': 271.6818414071356, '1050': 271.87488288967614, '226': 272.3283153042763, '2115': 272.884102718497, '64746': 272.9016054935477, '5883': 273.5247423572875, '10227': 276.74148283312235, '26036': 276.90931351708, '25839': 278.4835405889424, '10668': 282.0882377950404, '6194': 282.8152414636893, '4775': 284.03687931528344, '3682': 284.0756323957489, '2048': 286.2114811401823, '9801': 286.61053146444834, '23224': 286.97572608400804, '2264': 287.114960398912, '1846': 287.2711225661691, '8573': 287.8229494872216, '10589': 288.03781619787446, '5427': 288.41071884096647, '6696': 289.0679243722167, '4817': 289.1838216537197, '5899': 289.3249297338057, '2288': 289.40545950531384, '51160': 291.56594412563254, '3122': 292.2459117675861, '8826': 292.3123243345142, '840': 296.93697300354256, '79921': 298.4777063305921, '6908': 299.0999063117409, '5925': 300.9461928323634, '2771': 302.04458733881575, '4609': 302.8530576615638, '93487': 304.10422876480254, '80349': 304.98494591282895, '890': 305.3418575861588, '16': 305.60714589397764, '5777': 306.3399838209768, '63933': 306.90160549354766, '64781': 309.44817307894755, '1906': 309.6590792964322, '1070': 312.0193765402327, '3300': 314.5475044679908, '5300': 315.92343072166005, '128': 316.5951154173838, '2185': 317.7608593733552, '6009': 318.26867387828946, '2523': 319.47525764271256, '10972': 320.9428072618928, '994': 321.53979644445855, '899': 321.6105314644483, '23244': 322.4238991629556, '9124': 322.8923856647269, '23338': 323.344881196736, '23647': 324.16048462031875, '10681': 324.8220126046306, '9552': 328.28221616017197, '4794': 328.71382334665986, '9276': 328.9160846580212, '10285': 329.25361979111835, '10644': 329.4238991629556, '23443': 329.47338387753047, '4582': 329.84816028580457, '1831': 330.09068648292003, '11188': 330.7817477188765, '9650': 331.7288774338309, '3280': 333.3855080423835, '9943': 333.8278468629808, '10915': 334.12850268079455, '54541': 334.29727024734314, '54623': 334.46020355554145, '1891': 334.59605229997476, '3066': 335.60714589397764, '5058': 335.75898560817313, '5986': 338.71101269888675, '7015': 339.1076143352731, '9847': 339.7017928700659, '29103': 340.0141172045801, '9212': 340.1294395633855, '11065': 340.3821224719129, '28969': 342.8545694668524, '5261': 344.9151477754302, '65123': 344.99229197646775, '23321': 345.3768631362603, '55608': 346.1450685732542, '2736': 346.3904054181427, '1677': 346.48109190106277, '2852': 346.9763010067056, '6709': 347.65418192067295, '29937': 347.97572608400804, '7398': 348.08823779504047, '5710': 348.63141980996966, '8349': 349.7565369202935, '23512': 350.1905927946609, '3553': 350.3157099049848, '51021': 350.320607280744, '9533': 350.6381909509109, '5880': 350.64496209185216, '808': 351.60037475303636, '60493': 352.4447875084769, '1454': 353.9379098861336, '85377': 354.0829784593878, '4303': 354.6255855516194, '5627': 355.4786432131831, '3312': 355.8767566548582, '5373': 356.49520910564286, '1662': 357.82013883944853, '2920': 359.7608593733552, '8851': 360.72304317548065, '7485': 361.23518013347666, '57215': 362.08636402985843, '1017': 362.7866450946357, '10362': 362.9582233089573, '5357': 364.164445113487, '22926': 364.487863042004, '5583': 365.19642705301123, '5898': 365.8220126046306, '5223': 367.4558811024798, '51056': 367.6329316152582, '8607': 368.67751895407395, '80204': 369.72455498076926, '1845': 370.72981431642194, '2063': 370.8395153796813, '54512': 371.4238991629556, '23588': 371.47864321318315, '29082': 372.2895622238108, '5111': 372.75163954453427, '949': 373.11928285197365, '8061': 373.52661612246953, '23530': 374.115897281503, '5440': 375.65569372596156, '7319': 375.7405459505314, '23326': 375.8118558932187, '7168': 375.8724342017965, '23670': 375.97478920141697, '6390': 376.0882377950404, '11284': 376.42051359248495, '22889': 378.7851332893471, '79071': 381.6397027561994, '10953': 381.67262157831476, '64422': 382.6643386320849, '9531': 387.4389532501267, '57761': 387.7202325277076, '3480': 387.9093135170799, '66008': 390.3006558178137, '7994': 390.35842347861853, '6804': 390.91947022849183, '80758': 391.1048036875001, '4605': 391.19455328782914, '1870': 391.37967378403323, '2956': 391.6430883266701, '6500': 391.9539008558957, '23011': 392.57667575974205, '9712': 392.9326505504809, '10670': 393.2022613113614, '6347': 394.11834596938263, '7106': 394.4733838775305, '90861': 395.2585171668776, '3251': 395.3049782708753, '57804': 395.8801422253288, '5715': 396.5945404946862, '124583': 398.03932800316306, '3098': 400.32248104592605, '9016': 400.5238054746964, '51719': 400.6824163298332, '79850': 400.7328379269991, '154': 401.1512647914979, '9467': 401.6649135547824, '7358': 404.24836045546573, '4172': 405.2904991064018, '6772': 405.93301251037434, '11044': 406.40056212955466, '51335': 406.96463249000516, '8321': 407.41806490460533, '2770': 407.79529000075905, '8800': 407.87056043661454, '5699': 408.42332424025795, '5696': 408.6188144106781, '3693': 409.4506217668271, '51031': 409.4626522434211, '5708': 410.0135422818825, '6774': 410.257005361589, '1788': 410.6726215783147, '6777': 411.16970444913954, '2263': 412.39134230073375, '5721': 412.44759815624997, '5921': 412.5038540117661, '8446': 412.6882505881834}
gene_expression_filter = np.array(gene_expression[list(selected_gene.keys())],dtype="float32")
cell_viability = np.array(cell_viability,dtype="float32")
dtest = xgb.DMatrix(gene_expression_filter,cell_viability)

#Loading model
loaded_model = pickle.load(open("model_adjust\Achilles-L1000-144h_XGBoost_1.dat","rb"))
print("Model Loaded Successfully!")

#Predicting the cell viability
cl_prediction = loaded_model.predict(dtest)
print('Test_Best_Pearson:',sc.stats.pearsonr(cl_prediction,cell_viability))
print("MSE_Test: %.3f" % (mean_squared_error(cl_prediction,cell_viability)))

Model Loaded Successfully!
Test_Best_Pearson: (0.20467627, 1.1837302080589742e-86)
MSE_Test: 2.007


In [4]:
#Testing the CTRP-L1000-24h data by using the Achilles-L1000-144h model
import pandas as pd
import numpy as np
import warnings
import xgboost as xgb
import pickle
import scipy as sc
from sklearn.metrics import mean_squared_error
pd.set_option('display.max_columns',15)
warnings.filterwarnings("ignore")

#Retrieving the data
def getData():
    #1.Reading the data
    sig_viability_info = pd.read_table(r"individual_validation\LINCS_CTRP_withoutdose_24h.txt",sep="	",dtype=str)
    sig_viability_info_filter = sig_viability_info[['signature','cell_line','drug','dose','time','cell_viability']]
    sig_GE_info = pd.read_csv(r"individual_validation\LINCS_gene_expression_24h.csv",sep=",",dtype=str)
    sig_GE_info_filter = sig_GE_info[:]

    #2.Combining the two datasets
    sig_viability = pd.merge(sig_viability_info_filter,sig_GE_info_filter,left_on='signature',right_on='cid')
    select_sig_viavility2 = pd.DataFrame(sig_viability).loc[:,'5720':]
    select_sig_viavility_arr = select_sig_viavility2.copy()
    
    #3.Getting the cell viability value
    viability_value = pd.DataFrame(sig_viability).loc[:,'cell_viability']
    
    #4.Return the gene expression matrix and cell viability value
    return select_sig_viavility_arr,viability_value
    
#Retrieving the data
gene_expression,cell_viability = getData()

#Selecting the genes
selected_gene = {'26136': 1.4156162167257094, '8870': 6.753151349822873, '1514': 7.943744144483807, '2778': 10.606209011386643, '8624': 15.078081083628545, '55324': 16.865663060855262, '6676': 20.596989182565782, '11041': 28.606209011386646, '2745': 28.990780171179136, '29928': 30.29388467687246, '1062': 41.043650456224704, '10057': 42.212418022773285, '23210': 42.58438378327429, '3206': 43.53734775657896, '4690': 51.090686482920034, '58497': 53.419001787196336, '4216': 54.043650456224704, '51070': 61.68767566548581, '9448': 62.13095136867411, '1509': 62.37780001885124, '51005': 63.081466654099174, '1052': 64.55917298469132, '9928': 66.2284089925354, '5829': 66.98156034235828, '2767': 67.1435567679656, '22883': 69.25023422064776, '5927': 74.32154416333505, '79947': 74.75315134982289, '11344': 74.80940720533907, '664': 76.35352610285928, '80212': 76.49953155870449, '9903': 77.69689549430669, '9817': 78.41561621672571, '3930': 82.03104505693322, '4616': 82.94712971495443, '8520': 84.97817477188765, '6919': 85.67845583666495, '5747': 86.63386849784924, '10813': 92.58776935374492, '9217': 96.78081083628545, '7158': 101.51213695799598, '22823': 103.5431820149292, '29911': 104.79003066510631, '5720': 106.35258922026827, '11004': 107.44666127365892, '30': 110.88165403061737, '501': 111.35936036120953, '1983': 114.06209011386643, '4850': 116.85982880250502, '8996': 119.27450813663971, '23139': 119.57177838398279, '9183': 120.91853334590081, '4793': 121.44327570318829, '39': 124.67845583666495, '9797': 126.33753513309716, '11168': 126.39040541814273, '2274': 127.43160718648784, '10099': 133.56839281351216, '5997': 133.81617834628034, '9133': 134.52135678681685, '3775': 134.60037475303642, '7077': 134.81185589321868, '5993': 135.4442125857793, '9375': 136.06209011386642, '8884': 136.12173153985327, '25825': 138.10574057009111, '211': 138.32248104592608, '23223': 140.13188825126514, '6119': 142.44666127365892, '23131': 146.22502342206477, '3988': 147.04703602669534, '55127': 147.31232433451416, '55556': 149.8429009501519, '51203': 151.03198193952423, '3725': 151.0713099426873, '466': 152.43744144483807, '56654': 154.5407333270496, '10682': 160.85982880250504, '6117': 169.14939102631584, '230': 169.24778553276815, '5836': 172.78758197722672, '3028': 173.8176901515689, '6832': 174.55672429681172, '7153': 176.79680180604757, '891': 177.1285026807945, '332': 178.52567923987849, '10559': 180.9446810270748, '9924': 181.67262157831473, '1026': 183.79435311816798, '1738': 185.21580359324392, '10489': 185.21918916371453, '5290': 187.12173153985327, '148022': 188.35842347861853, '51097': 191.28372796546057, '7264': 193.76330806123477, '91949': 194.54750446799085, '4927': 195.2250234220648, '25793': 196.79003066510631, '2146': 198.23762882135628, '7866': 198.60376032350706, '375346': 199.64985946761135, '27346': 200.0208883455213, '26227': 201.6566306085526, '10775': 202.7303892391195, '2946': 204.34581807932702, '4891': 205.06547568433706, '7088': 205.20319819395243, '1021': 205.27544501923074, '5601': 205.31477302239378, '1759': 205.5416702096406, '51026': 205.69106123595643, '1027': 208.87920534273778, '81533': 211.166318878669, '392': 212.08391534197878, '3329': 212.43405587436746, '147179': 212.4457243910679, '10494': 213.77403969534421, '6182': 217.03198193952423, '8550': 220.69444680642707, '7048': 221.24439996229754, '3895': 221.53396218610834, '701': 223.15277659678648, '965': 223.47525764271253, '11200': 226.29482155946351, '9653': 227.37290264309203, '3385': 229.35014053238865, '9833': 231.92285579896247, '5480': 232.61636572279852, '2037': 232.77591346052625, '84722': 233.64985946761138, '1399': 234.114960398912, '4846': 234.52135678681682, '8878': 235.13678562702435, '84617': 235.27544501923074, '51001': 236.82294948722154, '23212': 238.11683416409403, '6616': 239.23179456300605, '58533': 240.15522528466607, '6790': 240.25174602593634, '23386': 240.6968954943067, '80347': 241.00921982882087, '30001': 241.2133549053643, '8985': 241.42483604554658, '10451': 241.70705220571855, '9868': 242.74054595053138, '7043': 248.13678562702435, '9275': 250.2745081366397, '79143': 253.16048462031873, '8440': 254.62371178643735, '993': 256.6581424138412, '8508': 257.0839153419788, '4893': 259.58193509539467, '9517': 260.07224682527834, '55148': 260.24778553276815, '4860': 260.4694233843623, '27242': 263.6658504373735, '10523': 266.231794563006, '10513': 267.46754961918026, '256364': 268.60959458185727, '6944': 269.10574057009114, '7165': 270.3476918445091, '2017': 271.6818414071356, '1050': 271.87488288967614, '226': 272.3283153042763, '2115': 272.884102718497, '64746': 272.9016054935477, '5883': 273.5247423572875, '10227': 276.74148283312235, '26036': 276.90931351708, '25839': 278.4835405889424, '10668': 282.0882377950404, '6194': 282.8152414636893, '4775': 284.03687931528344, '3682': 284.0756323957489, '2048': 286.2114811401823, '9801': 286.61053146444834, '23224': 286.97572608400804, '2264': 287.114960398912, '1846': 287.2711225661691, '8573': 287.8229494872216, '10589': 288.03781619787446, '5427': 288.41071884096647, '6696': 289.0679243722167, '4817': 289.1838216537197, '5899': 289.3249297338057, '2288': 289.40545950531384, '51160': 291.56594412563254, '3122': 292.2459117675861, '8826': 292.3123243345142, '840': 296.93697300354256, '79921': 298.4777063305921, '6908': 299.0999063117409, '5925': 300.9461928323634, '2771': 302.04458733881575, '4609': 302.8530576615638, '93487': 304.10422876480254, '80349': 304.98494591282895, '890': 305.3418575861588, '16': 305.60714589397764, '5777': 306.3399838209768, '63933': 306.90160549354766, '64781': 309.44817307894755, '1906': 309.6590792964322, '1070': 312.0193765402327, '3300': 314.5475044679908, '5300': 315.92343072166005, '128': 316.5951154173838, '2185': 317.7608593733552, '6009': 318.26867387828946, '2523': 319.47525764271256, '10972': 320.9428072618928, '994': 321.53979644445855, '899': 321.6105314644483, '23244': 322.4238991629556, '9124': 322.8923856647269, '23338': 323.344881196736, '23647': 324.16048462031875, '10681': 324.8220126046306, '9552': 328.28221616017197, '4794': 328.71382334665986, '9276': 328.9160846580212, '10285': 329.25361979111835, '10644': 329.4238991629556, '23443': 329.47338387753047, '4582': 329.84816028580457, '1831': 330.09068648292003, '11188': 330.7817477188765, '9650': 331.7288774338309, '3280': 333.3855080423835, '9943': 333.8278468629808, '10915': 334.12850268079455, '54541': 334.29727024734314, '54623': 334.46020355554145, '1891': 334.59605229997476, '3066': 335.60714589397764, '5058': 335.75898560817313, '5986': 338.71101269888675, '7015': 339.1076143352731, '9847': 339.7017928700659, '29103': 340.0141172045801, '9212': 340.1294395633855, '11065': 340.3821224719129, '28969': 342.8545694668524, '5261': 344.9151477754302, '65123': 344.99229197646775, '23321': 345.3768631362603, '55608': 346.1450685732542, '2736': 346.3904054181427, '1677': 346.48109190106277, '2852': 346.9763010067056, '6709': 347.65418192067295, '29937': 347.97572608400804, '7398': 348.08823779504047, '5710': 348.63141980996966, '8349': 349.7565369202935, '23512': 350.1905927946609, '3553': 350.3157099049848, '51021': 350.320607280744, '9533': 350.6381909509109, '5880': 350.64496209185216, '808': 351.60037475303636, '60493': 352.4447875084769, '1454': 353.9379098861336, '85377': 354.0829784593878, '4303': 354.6255855516194, '5627': 355.4786432131831, '3312': 355.8767566548582, '5373': 356.49520910564286, '1662': 357.82013883944853, '2920': 359.7608593733552, '8851': 360.72304317548065, '7485': 361.23518013347666, '57215': 362.08636402985843, '1017': 362.7866450946357, '10362': 362.9582233089573, '5357': 364.164445113487, '22926': 364.487863042004, '5583': 365.19642705301123, '5898': 365.8220126046306, '5223': 367.4558811024798, '51056': 367.6329316152582, '8607': 368.67751895407395, '80204': 369.72455498076926, '1845': 370.72981431642194, '2063': 370.8395153796813, '54512': 371.4238991629556, '23588': 371.47864321318315, '29082': 372.2895622238108, '5111': 372.75163954453427, '949': 373.11928285197365, '8061': 373.52661612246953, '23530': 374.115897281503, '5440': 375.65569372596156, '7319': 375.7405459505314, '23326': 375.8118558932187, '7168': 375.8724342017965, '23670': 375.97478920141697, '6390': 376.0882377950404, '11284': 376.42051359248495, '22889': 378.7851332893471, '79071': 381.6397027561994, '10953': 381.67262157831476, '64422': 382.6643386320849, '9531': 387.4389532501267, '57761': 387.7202325277076, '3480': 387.9093135170799, '66008': 390.3006558178137, '7994': 390.35842347861853, '6804': 390.91947022849183, '80758': 391.1048036875001, '4605': 391.19455328782914, '1870': 391.37967378403323, '2956': 391.6430883266701, '6500': 391.9539008558957, '23011': 392.57667575974205, '9712': 392.9326505504809, '10670': 393.2022613113614, '6347': 394.11834596938263, '7106': 394.4733838775305, '90861': 395.2585171668776, '3251': 395.3049782708753, '57804': 395.8801422253288, '5715': 396.5945404946862, '124583': 398.03932800316306, '3098': 400.32248104592605, '9016': 400.5238054746964, '51719': 400.6824163298332, '79850': 400.7328379269991, '154': 401.1512647914979, '9467': 401.6649135547824, '7358': 404.24836045546573, '4172': 405.2904991064018, '6772': 405.93301251037434, '11044': 406.40056212955466, '51335': 406.96463249000516, '8321': 407.41806490460533, '2770': 407.79529000075905, '8800': 407.87056043661454, '5699': 408.42332424025795, '5696': 408.6188144106781, '3693': 409.4506217668271, '51031': 409.4626522434211, '5708': 410.0135422818825, '6774': 410.257005361589, '1788': 410.6726215783147, '6777': 411.16970444913954, '2263': 412.39134230073375, '5721': 412.44759815624997, '5921': 412.5038540117661, '8446': 412.6882505881834}
gene_expression_filter = np.array(gene_expression[list(selected_gene.keys())],dtype="float32")
cell_viability = np.array(cell_viability,dtype="float32")
dtest = xgb.DMatrix(gene_expression_filter,cell_viability)

#Loading model
loaded_model = pickle.load(open("model_adjust\Achilles-L1000-144h_XGBoost_1.dat","rb"))
print("Model Loaded Successfully!")

#Predicting the cell viability
cl_prediction = loaded_model.predict(dtest)
print('Test_Best_Pearson:',sc.stats.pearsonr(cl_prediction,cell_viability))
print("MSE_Test: %.3f" % (mean_squared_error(cl_prediction,cell_viability)))

Model Loaded Successfully!
Test_Best_Pearson: (0.6271799, 0.0)
MSE_Test: 2.755


In [5]:
#Testing the Achilles-L1000-96h data by using the Achilles-L1000-144h model
import pandas as pd
import numpy as np
import warnings
import xgboost as xgb
import pickle
import scipy as sc
from sklearn.metrics import mean_squared_error
pd.set_option('display.max_columns',15)
warnings.filterwarnings("ignore")

#Retrieving the data
def getData():
    #1.Reading the data
    sig_viability_info = pd.read_table(r"generate\Achilles_L1000_PhaseI_96h_unique.txt",sep="	",dtype=str)
    sig_viability_info_filter = sig_viability_info[['signature','cell_line','drug','pert_type','time','cell_viability']]
    sig_GE_info = pd.read_csv(r"generate\Achilles_L1000_PhaseI_gene_expression_96h.csv",sep=",",dtype=str)
    sig_GE_info_filter = sig_GE_info[:]

    #2.Combining the two datasets
    sig_viability = pd.merge(sig_viability_info_filter,sig_GE_info_filter,left_on='signature',right_on='cid')
    select_sig_viavility2 = pd.DataFrame(sig_viability).loc[:,'5720':]
    select_sig_viavility_arr = select_sig_viavility2.copy()
    
    #3.Getting the cell viability value
    viability_value = pd.DataFrame(sig_viability).loc[:,'cell_viability']
    
    #4.Return the gene expression matrix and cell viability value
    return select_sig_viavility_arr,viability_value
    
#Retrieving the data
gene_expression,cell_viability = getData()

#Selecting the genes
selected_gene = {'26136': 1.4156162167257094, '8870': 6.753151349822873, '1514': 7.943744144483807, '2778': 10.606209011386643, '8624': 15.078081083628545, '55324': 16.865663060855262, '6676': 20.596989182565782, '11041': 28.606209011386646, '2745': 28.990780171179136, '29928': 30.29388467687246, '1062': 41.043650456224704, '10057': 42.212418022773285, '23210': 42.58438378327429, '3206': 43.53734775657896, '4690': 51.090686482920034, '58497': 53.419001787196336, '4216': 54.043650456224704, '51070': 61.68767566548581, '9448': 62.13095136867411, '1509': 62.37780001885124, '51005': 63.081466654099174, '1052': 64.55917298469132, '9928': 66.2284089925354, '5829': 66.98156034235828, '2767': 67.1435567679656, '22883': 69.25023422064776, '5927': 74.32154416333505, '79947': 74.75315134982289, '11344': 74.80940720533907, '664': 76.35352610285928, '80212': 76.49953155870449, '9903': 77.69689549430669, '9817': 78.41561621672571, '3930': 82.03104505693322, '4616': 82.94712971495443, '8520': 84.97817477188765, '6919': 85.67845583666495, '5747': 86.63386849784924, '10813': 92.58776935374492, '9217': 96.78081083628545, '7158': 101.51213695799598, '22823': 103.5431820149292, '29911': 104.79003066510631, '5720': 106.35258922026827, '11004': 107.44666127365892, '30': 110.88165403061737, '501': 111.35936036120953, '1983': 114.06209011386643, '4850': 116.85982880250502, '8996': 119.27450813663971, '23139': 119.57177838398279, '9183': 120.91853334590081, '4793': 121.44327570318829, '39': 124.67845583666495, '9797': 126.33753513309716, '11168': 126.39040541814273, '2274': 127.43160718648784, '10099': 133.56839281351216, '5997': 133.81617834628034, '9133': 134.52135678681685, '3775': 134.60037475303642, '7077': 134.81185589321868, '5993': 135.4442125857793, '9375': 136.06209011386642, '8884': 136.12173153985327, '25825': 138.10574057009111, '211': 138.32248104592608, '23223': 140.13188825126514, '6119': 142.44666127365892, '23131': 146.22502342206477, '3988': 147.04703602669534, '55127': 147.31232433451416, '55556': 149.8429009501519, '51203': 151.03198193952423, '3725': 151.0713099426873, '466': 152.43744144483807, '56654': 154.5407333270496, '10682': 160.85982880250504, '6117': 169.14939102631584, '230': 169.24778553276815, '5836': 172.78758197722672, '3028': 173.8176901515689, '6832': 174.55672429681172, '7153': 176.79680180604757, '891': 177.1285026807945, '332': 178.52567923987849, '10559': 180.9446810270748, '9924': 181.67262157831473, '1026': 183.79435311816798, '1738': 185.21580359324392, '10489': 185.21918916371453, '5290': 187.12173153985327, '148022': 188.35842347861853, '51097': 191.28372796546057, '7264': 193.76330806123477, '91949': 194.54750446799085, '4927': 195.2250234220648, '25793': 196.79003066510631, '2146': 198.23762882135628, '7866': 198.60376032350706, '375346': 199.64985946761135, '27346': 200.0208883455213, '26227': 201.6566306085526, '10775': 202.7303892391195, '2946': 204.34581807932702, '4891': 205.06547568433706, '7088': 205.20319819395243, '1021': 205.27544501923074, '5601': 205.31477302239378, '1759': 205.5416702096406, '51026': 205.69106123595643, '1027': 208.87920534273778, '81533': 211.166318878669, '392': 212.08391534197878, '3329': 212.43405587436746, '147179': 212.4457243910679, '10494': 213.77403969534421, '6182': 217.03198193952423, '8550': 220.69444680642707, '7048': 221.24439996229754, '3895': 221.53396218610834, '701': 223.15277659678648, '965': 223.47525764271253, '11200': 226.29482155946351, '9653': 227.37290264309203, '3385': 229.35014053238865, '9833': 231.92285579896247, '5480': 232.61636572279852, '2037': 232.77591346052625, '84722': 233.64985946761138, '1399': 234.114960398912, '4846': 234.52135678681682, '8878': 235.13678562702435, '84617': 235.27544501923074, '51001': 236.82294948722154, '23212': 238.11683416409403, '6616': 239.23179456300605, '58533': 240.15522528466607, '6790': 240.25174602593634, '23386': 240.6968954943067, '80347': 241.00921982882087, '30001': 241.2133549053643, '8985': 241.42483604554658, '10451': 241.70705220571855, '9868': 242.74054595053138, '7043': 248.13678562702435, '9275': 250.2745081366397, '79143': 253.16048462031873, '8440': 254.62371178643735, '993': 256.6581424138412, '8508': 257.0839153419788, '4893': 259.58193509539467, '9517': 260.07224682527834, '55148': 260.24778553276815, '4860': 260.4694233843623, '27242': 263.6658504373735, '10523': 266.231794563006, '10513': 267.46754961918026, '256364': 268.60959458185727, '6944': 269.10574057009114, '7165': 270.3476918445091, '2017': 271.6818414071356, '1050': 271.87488288967614, '226': 272.3283153042763, '2115': 272.884102718497, '64746': 272.9016054935477, '5883': 273.5247423572875, '10227': 276.74148283312235, '26036': 276.90931351708, '25839': 278.4835405889424, '10668': 282.0882377950404, '6194': 282.8152414636893, '4775': 284.03687931528344, '3682': 284.0756323957489, '2048': 286.2114811401823, '9801': 286.61053146444834, '23224': 286.97572608400804, '2264': 287.114960398912, '1846': 287.2711225661691, '8573': 287.8229494872216, '10589': 288.03781619787446, '5427': 288.41071884096647, '6696': 289.0679243722167, '4817': 289.1838216537197, '5899': 289.3249297338057, '2288': 289.40545950531384, '51160': 291.56594412563254, '3122': 292.2459117675861, '8826': 292.3123243345142, '840': 296.93697300354256, '79921': 298.4777063305921, '6908': 299.0999063117409, '5925': 300.9461928323634, '2771': 302.04458733881575, '4609': 302.8530576615638, '93487': 304.10422876480254, '80349': 304.98494591282895, '890': 305.3418575861588, '16': 305.60714589397764, '5777': 306.3399838209768, '63933': 306.90160549354766, '64781': 309.44817307894755, '1906': 309.6590792964322, '1070': 312.0193765402327, '3300': 314.5475044679908, '5300': 315.92343072166005, '128': 316.5951154173838, '2185': 317.7608593733552, '6009': 318.26867387828946, '2523': 319.47525764271256, '10972': 320.9428072618928, '994': 321.53979644445855, '899': 321.6105314644483, '23244': 322.4238991629556, '9124': 322.8923856647269, '23338': 323.344881196736, '23647': 324.16048462031875, '10681': 324.8220126046306, '9552': 328.28221616017197, '4794': 328.71382334665986, '9276': 328.9160846580212, '10285': 329.25361979111835, '10644': 329.4238991629556, '23443': 329.47338387753047, '4582': 329.84816028580457, '1831': 330.09068648292003, '11188': 330.7817477188765, '9650': 331.7288774338309, '3280': 333.3855080423835, '9943': 333.8278468629808, '10915': 334.12850268079455, '54541': 334.29727024734314, '54623': 334.46020355554145, '1891': 334.59605229997476, '3066': 335.60714589397764, '5058': 335.75898560817313, '5986': 338.71101269888675, '7015': 339.1076143352731, '9847': 339.7017928700659, '29103': 340.0141172045801, '9212': 340.1294395633855, '11065': 340.3821224719129, '28969': 342.8545694668524, '5261': 344.9151477754302, '65123': 344.99229197646775, '23321': 345.3768631362603, '55608': 346.1450685732542, '2736': 346.3904054181427, '1677': 346.48109190106277, '2852': 346.9763010067056, '6709': 347.65418192067295, '29937': 347.97572608400804, '7398': 348.08823779504047, '5710': 348.63141980996966, '8349': 349.7565369202935, '23512': 350.1905927946609, '3553': 350.3157099049848, '51021': 350.320607280744, '9533': 350.6381909509109, '5880': 350.64496209185216, '808': 351.60037475303636, '60493': 352.4447875084769, '1454': 353.9379098861336, '85377': 354.0829784593878, '4303': 354.6255855516194, '5627': 355.4786432131831, '3312': 355.8767566548582, '5373': 356.49520910564286, '1662': 357.82013883944853, '2920': 359.7608593733552, '8851': 360.72304317548065, '7485': 361.23518013347666, '57215': 362.08636402985843, '1017': 362.7866450946357, '10362': 362.9582233089573, '5357': 364.164445113487, '22926': 364.487863042004, '5583': 365.19642705301123, '5898': 365.8220126046306, '5223': 367.4558811024798, '51056': 367.6329316152582, '8607': 368.67751895407395, '80204': 369.72455498076926, '1845': 370.72981431642194, '2063': 370.8395153796813, '54512': 371.4238991629556, '23588': 371.47864321318315, '29082': 372.2895622238108, '5111': 372.75163954453427, '949': 373.11928285197365, '8061': 373.52661612246953, '23530': 374.115897281503, '5440': 375.65569372596156, '7319': 375.7405459505314, '23326': 375.8118558932187, '7168': 375.8724342017965, '23670': 375.97478920141697, '6390': 376.0882377950404, '11284': 376.42051359248495, '22889': 378.7851332893471, '79071': 381.6397027561994, '10953': 381.67262157831476, '64422': 382.6643386320849, '9531': 387.4389532501267, '57761': 387.7202325277076, '3480': 387.9093135170799, '66008': 390.3006558178137, '7994': 390.35842347861853, '6804': 390.91947022849183, '80758': 391.1048036875001, '4605': 391.19455328782914, '1870': 391.37967378403323, '2956': 391.6430883266701, '6500': 391.9539008558957, '23011': 392.57667575974205, '9712': 392.9326505504809, '10670': 393.2022613113614, '6347': 394.11834596938263, '7106': 394.4733838775305, '90861': 395.2585171668776, '3251': 395.3049782708753, '57804': 395.8801422253288, '5715': 396.5945404946862, '124583': 398.03932800316306, '3098': 400.32248104592605, '9016': 400.5238054746964, '51719': 400.6824163298332, '79850': 400.7328379269991, '154': 401.1512647914979, '9467': 401.6649135547824, '7358': 404.24836045546573, '4172': 405.2904991064018, '6772': 405.93301251037434, '11044': 406.40056212955466, '51335': 406.96463249000516, '8321': 407.41806490460533, '2770': 407.79529000075905, '8800': 407.87056043661454, '5699': 408.42332424025795, '5696': 408.6188144106781, '3693': 409.4506217668271, '51031': 409.4626522434211, '5708': 410.0135422818825, '6774': 410.257005361589, '1788': 410.6726215783147, '6777': 411.16970444913954, '2263': 412.39134230073375, '5721': 412.44759815624997, '5921': 412.5038540117661, '8446': 412.6882505881834}
gene_expression_filter = np.array(gene_expression[list(selected_gene.keys())],dtype="float32")
cell_viability = np.array(cell_viability,dtype="float32")
dtest = xgb.DMatrix(gene_expression_filter,cell_viability)

#Loading model
loaded_model = pickle.load(open("model_adjust\Achilles-L1000-144h_XGBoost_1.dat","rb"))
print("Model Loaded Successfully!")

#Predicting the cell viability
cl_prediction = loaded_model.predict(dtest)
print('Test_Best_Pearson:',sc.stats.pearsonr(cl_prediction,cell_viability))
print("MSE_Test: %.3f" % (mean_squared_error(cl_prediction,cell_viability)))

Model Loaded Successfully!
Test_Best_Pearson: (0.3398263, 0.0)
MSE_Test: 2.038


In [6]:
#Testing the Achilles-L1000-120h data by using the Achilles-L1000-144h model
import pandas as pd
import numpy as np
import warnings
import xgboost as xgb
import pickle
import scipy as sc
from sklearn.metrics import mean_squared_error
pd.set_option('display.max_columns',15)
warnings.filterwarnings("ignore")

#Retrieving the data
def getData():
    #1.Reading the data
    sig_viability_info = pd.read_table(r"generate\Achilles_L1000_PhaseI_120h_unique.txt",sep="	",dtype=str)
    sig_viability_info_filter = sig_viability_info[['signature','cell_line','drug','pert_type','time','cell_viability']]
    sig_GE_info = pd.read_csv(r"generate\Achilles_L1000_PhaseI_gene_expression_120h.csv",sep=",",dtype=str)
    sig_GE_info_filter = sig_GE_info[:]

    #2.Combining the two datasets
    sig_viability = pd.merge(sig_viability_info_filter,sig_GE_info_filter,left_on='signature',right_on='cid')
    select_sig_viavility2 = pd.DataFrame(sig_viability).loc[:,'5720':]
    select_sig_viavility_arr = select_sig_viavility2.copy()
    
    #3.Getting the cell viability value
    viability_value = pd.DataFrame(sig_viability).loc[:,'cell_viability']
    
    #4.Return the gene expression matrix and cell viability value
    return select_sig_viavility_arr,viability_value
    
#Retrieving the data
gene_expression,cell_viability = getData()

#Selecting the genes
selected_gene = {'26136': 1.4156162167257094, '8870': 6.753151349822873, '1514': 7.943744144483807, '2778': 10.606209011386643, '8624': 15.078081083628545, '55324': 16.865663060855262, '6676': 20.596989182565782, '11041': 28.606209011386646, '2745': 28.990780171179136, '29928': 30.29388467687246, '1062': 41.043650456224704, '10057': 42.212418022773285, '23210': 42.58438378327429, '3206': 43.53734775657896, '4690': 51.090686482920034, '58497': 53.419001787196336, '4216': 54.043650456224704, '51070': 61.68767566548581, '9448': 62.13095136867411, '1509': 62.37780001885124, '51005': 63.081466654099174, '1052': 64.55917298469132, '9928': 66.2284089925354, '5829': 66.98156034235828, '2767': 67.1435567679656, '22883': 69.25023422064776, '5927': 74.32154416333505, '79947': 74.75315134982289, '11344': 74.80940720533907, '664': 76.35352610285928, '80212': 76.49953155870449, '9903': 77.69689549430669, '9817': 78.41561621672571, '3930': 82.03104505693322, '4616': 82.94712971495443, '8520': 84.97817477188765, '6919': 85.67845583666495, '5747': 86.63386849784924, '10813': 92.58776935374492, '9217': 96.78081083628545, '7158': 101.51213695799598, '22823': 103.5431820149292, '29911': 104.79003066510631, '5720': 106.35258922026827, '11004': 107.44666127365892, '30': 110.88165403061737, '501': 111.35936036120953, '1983': 114.06209011386643, '4850': 116.85982880250502, '8996': 119.27450813663971, '23139': 119.57177838398279, '9183': 120.91853334590081, '4793': 121.44327570318829, '39': 124.67845583666495, '9797': 126.33753513309716, '11168': 126.39040541814273, '2274': 127.43160718648784, '10099': 133.56839281351216, '5997': 133.81617834628034, '9133': 134.52135678681685, '3775': 134.60037475303642, '7077': 134.81185589321868, '5993': 135.4442125857793, '9375': 136.06209011386642, '8884': 136.12173153985327, '25825': 138.10574057009111, '211': 138.32248104592608, '23223': 140.13188825126514, '6119': 142.44666127365892, '23131': 146.22502342206477, '3988': 147.04703602669534, '55127': 147.31232433451416, '55556': 149.8429009501519, '51203': 151.03198193952423, '3725': 151.0713099426873, '466': 152.43744144483807, '56654': 154.5407333270496, '10682': 160.85982880250504, '6117': 169.14939102631584, '230': 169.24778553276815, '5836': 172.78758197722672, '3028': 173.8176901515689, '6832': 174.55672429681172, '7153': 176.79680180604757, '891': 177.1285026807945, '332': 178.52567923987849, '10559': 180.9446810270748, '9924': 181.67262157831473, '1026': 183.79435311816798, '1738': 185.21580359324392, '10489': 185.21918916371453, '5290': 187.12173153985327, '148022': 188.35842347861853, '51097': 191.28372796546057, '7264': 193.76330806123477, '91949': 194.54750446799085, '4927': 195.2250234220648, '25793': 196.79003066510631, '2146': 198.23762882135628, '7866': 198.60376032350706, '375346': 199.64985946761135, '27346': 200.0208883455213, '26227': 201.6566306085526, '10775': 202.7303892391195, '2946': 204.34581807932702, '4891': 205.06547568433706, '7088': 205.20319819395243, '1021': 205.27544501923074, '5601': 205.31477302239378, '1759': 205.5416702096406, '51026': 205.69106123595643, '1027': 208.87920534273778, '81533': 211.166318878669, '392': 212.08391534197878, '3329': 212.43405587436746, '147179': 212.4457243910679, '10494': 213.77403969534421, '6182': 217.03198193952423, '8550': 220.69444680642707, '7048': 221.24439996229754, '3895': 221.53396218610834, '701': 223.15277659678648, '965': 223.47525764271253, '11200': 226.29482155946351, '9653': 227.37290264309203, '3385': 229.35014053238865, '9833': 231.92285579896247, '5480': 232.61636572279852, '2037': 232.77591346052625, '84722': 233.64985946761138, '1399': 234.114960398912, '4846': 234.52135678681682, '8878': 235.13678562702435, '84617': 235.27544501923074, '51001': 236.82294948722154, '23212': 238.11683416409403, '6616': 239.23179456300605, '58533': 240.15522528466607, '6790': 240.25174602593634, '23386': 240.6968954943067, '80347': 241.00921982882087, '30001': 241.2133549053643, '8985': 241.42483604554658, '10451': 241.70705220571855, '9868': 242.74054595053138, '7043': 248.13678562702435, '9275': 250.2745081366397, '79143': 253.16048462031873, '8440': 254.62371178643735, '993': 256.6581424138412, '8508': 257.0839153419788, '4893': 259.58193509539467, '9517': 260.07224682527834, '55148': 260.24778553276815, '4860': 260.4694233843623, '27242': 263.6658504373735, '10523': 266.231794563006, '10513': 267.46754961918026, '256364': 268.60959458185727, '6944': 269.10574057009114, '7165': 270.3476918445091, '2017': 271.6818414071356, '1050': 271.87488288967614, '226': 272.3283153042763, '2115': 272.884102718497, '64746': 272.9016054935477, '5883': 273.5247423572875, '10227': 276.74148283312235, '26036': 276.90931351708, '25839': 278.4835405889424, '10668': 282.0882377950404, '6194': 282.8152414636893, '4775': 284.03687931528344, '3682': 284.0756323957489, '2048': 286.2114811401823, '9801': 286.61053146444834, '23224': 286.97572608400804, '2264': 287.114960398912, '1846': 287.2711225661691, '8573': 287.8229494872216, '10589': 288.03781619787446, '5427': 288.41071884096647, '6696': 289.0679243722167, '4817': 289.1838216537197, '5899': 289.3249297338057, '2288': 289.40545950531384, '51160': 291.56594412563254, '3122': 292.2459117675861, '8826': 292.3123243345142, '840': 296.93697300354256, '79921': 298.4777063305921, '6908': 299.0999063117409, '5925': 300.9461928323634, '2771': 302.04458733881575, '4609': 302.8530576615638, '93487': 304.10422876480254, '80349': 304.98494591282895, '890': 305.3418575861588, '16': 305.60714589397764, '5777': 306.3399838209768, '63933': 306.90160549354766, '64781': 309.44817307894755, '1906': 309.6590792964322, '1070': 312.0193765402327, '3300': 314.5475044679908, '5300': 315.92343072166005, '128': 316.5951154173838, '2185': 317.7608593733552, '6009': 318.26867387828946, '2523': 319.47525764271256, '10972': 320.9428072618928, '994': 321.53979644445855, '899': 321.6105314644483, '23244': 322.4238991629556, '9124': 322.8923856647269, '23338': 323.344881196736, '23647': 324.16048462031875, '10681': 324.8220126046306, '9552': 328.28221616017197, '4794': 328.71382334665986, '9276': 328.9160846580212, '10285': 329.25361979111835, '10644': 329.4238991629556, '23443': 329.47338387753047, '4582': 329.84816028580457, '1831': 330.09068648292003, '11188': 330.7817477188765, '9650': 331.7288774338309, '3280': 333.3855080423835, '9943': 333.8278468629808, '10915': 334.12850268079455, '54541': 334.29727024734314, '54623': 334.46020355554145, '1891': 334.59605229997476, '3066': 335.60714589397764, '5058': 335.75898560817313, '5986': 338.71101269888675, '7015': 339.1076143352731, '9847': 339.7017928700659, '29103': 340.0141172045801, '9212': 340.1294395633855, '11065': 340.3821224719129, '28969': 342.8545694668524, '5261': 344.9151477754302, '65123': 344.99229197646775, '23321': 345.3768631362603, '55608': 346.1450685732542, '2736': 346.3904054181427, '1677': 346.48109190106277, '2852': 346.9763010067056, '6709': 347.65418192067295, '29937': 347.97572608400804, '7398': 348.08823779504047, '5710': 348.63141980996966, '8349': 349.7565369202935, '23512': 350.1905927946609, '3553': 350.3157099049848, '51021': 350.320607280744, '9533': 350.6381909509109, '5880': 350.64496209185216, '808': 351.60037475303636, '60493': 352.4447875084769, '1454': 353.9379098861336, '85377': 354.0829784593878, '4303': 354.6255855516194, '5627': 355.4786432131831, '3312': 355.8767566548582, '5373': 356.49520910564286, '1662': 357.82013883944853, '2920': 359.7608593733552, '8851': 360.72304317548065, '7485': 361.23518013347666, '57215': 362.08636402985843, '1017': 362.7866450946357, '10362': 362.9582233089573, '5357': 364.164445113487, '22926': 364.487863042004, '5583': 365.19642705301123, '5898': 365.8220126046306, '5223': 367.4558811024798, '51056': 367.6329316152582, '8607': 368.67751895407395, '80204': 369.72455498076926, '1845': 370.72981431642194, '2063': 370.8395153796813, '54512': 371.4238991629556, '23588': 371.47864321318315, '29082': 372.2895622238108, '5111': 372.75163954453427, '949': 373.11928285197365, '8061': 373.52661612246953, '23530': 374.115897281503, '5440': 375.65569372596156, '7319': 375.7405459505314, '23326': 375.8118558932187, '7168': 375.8724342017965, '23670': 375.97478920141697, '6390': 376.0882377950404, '11284': 376.42051359248495, '22889': 378.7851332893471, '79071': 381.6397027561994, '10953': 381.67262157831476, '64422': 382.6643386320849, '9531': 387.4389532501267, '57761': 387.7202325277076, '3480': 387.9093135170799, '66008': 390.3006558178137, '7994': 390.35842347861853, '6804': 390.91947022849183, '80758': 391.1048036875001, '4605': 391.19455328782914, '1870': 391.37967378403323, '2956': 391.6430883266701, '6500': 391.9539008558957, '23011': 392.57667575974205, '9712': 392.9326505504809, '10670': 393.2022613113614, '6347': 394.11834596938263, '7106': 394.4733838775305, '90861': 395.2585171668776, '3251': 395.3049782708753, '57804': 395.8801422253288, '5715': 396.5945404946862, '124583': 398.03932800316306, '3098': 400.32248104592605, '9016': 400.5238054746964, '51719': 400.6824163298332, '79850': 400.7328379269991, '154': 401.1512647914979, '9467': 401.6649135547824, '7358': 404.24836045546573, '4172': 405.2904991064018, '6772': 405.93301251037434, '11044': 406.40056212955466, '51335': 406.96463249000516, '8321': 407.41806490460533, '2770': 407.79529000075905, '8800': 407.87056043661454, '5699': 408.42332424025795, '5696': 408.6188144106781, '3693': 409.4506217668271, '51031': 409.4626522434211, '5708': 410.0135422818825, '6774': 410.257005361589, '1788': 410.6726215783147, '6777': 411.16970444913954, '2263': 412.39134230073375, '5721': 412.44759815624997, '5921': 412.5038540117661, '8446': 412.6882505881834}
gene_expression_filter = np.array(gene_expression[list(selected_gene.keys())],dtype="float32")
cell_viability = np.array(cell_viability,dtype="float32")
dtest = xgb.DMatrix(gene_expression_filter,cell_viability)

#Loading model
loaded_model = pickle.load(open("model_adjust\Achilles-L1000-144h_XGBoost_1.dat","rb"))
print("Model Loaded Successfully!")

#Predicting the cell viability
cl_prediction = loaded_model.predict(dtest)
print('Test_Best_Pearson:',sc.stats.pearsonr(cl_prediction,cell_viability))
print("MSE_Test: %.3f" % (mean_squared_error(cl_prediction,cell_viability)))

Model Loaded Successfully!
Test_Best_Pearson: (0.2659624, 2.7036889965719246e-234)
MSE_Test: 1.873
